In [1]:
import requests
import os
import xml
from bs4 import BeautifulSoup
import pandas as pd
import re
from zipfile import ZipFile
import sqlite3
import openpyxl as xl

In [2]:
#Website and file type to download

Domain = 'http://mis.ercot.com'
url = 'http://mis.ercot.com/misapp/GetReports.do?reportTypeId=13061&reportTitle=Historical%20RTM%20Load%20Zone%20and%20Hub%20Prices&showHTMLView=&mimicKey'
filetype = 'zip'

In [3]:
#Get websites HTML, get all the filename and associated links

soup = BeautifulSoup(requests.get(url).text, 'html.parser')
file_list = soup.find_all(class_='labelOptional_ind')
link_list = soup.findAll('a', attrs={'href': re.compile("/misdownload/")}) 

In [4]:
#Slim down the information in the previous list and put them into new lists

link_name_list = []

for link in link_list:
    link_name_list.append(link.get('href'))

folder_name_list = []    
    
for class_ in file_list:
    folder_name_list.append(str(class_.next_element))

In [5]:
#create a df of filename and links 

data_dict = {'ZipFolderName':folder_name_list, 'DownLoadLink':link_name_list}
download_df = pd.DataFrame(data_dict)

In [6]:
#Download each zip from df that hasn't been downloaded already and save to specific folder in wd

zip_list = os.listdir(os.path.join(os.getcwd(),  'LMP_Zips'))
save_folder = 'LMP_Zips/'

for i in range(len(download_df)):
    file_name = download_df['ZipFolderName'][i]
    link_name = download_df['DownLoadLink'][i]
    
    if file_name not in zip_list:
        with open(save_folder + file_name, 'wb') as file:
            response = requests.get(Domain + link_name)
            file.write(response.content)


In [8]:
#extracts the xlsx from each zip and places in same directory

zip_path = os.path.join(os.getcwd(),  'LMP_Zips')

for file in os.listdir(zip_path):
    if '.zip' in file:
        with ZipFile(zip_path + '/' + file, 'r') as zipObj:
            zipObj.extractall(path=zip_path)

In [ ]:
#might use to delete old zip files after excel is extracted
zip_path = os.path.join(os.getcwd(),  'LMP_Zips')

for file in os.listdir(zip_path):
    if '.zip' in file:

In [9]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

In [10]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except (TypeError, NameError) as e:
        print(e)

In [43]:
def delete_rows(conn, delete_sql):
    ''' Delete rows from a sqlite db
    :param conn: Connection object
    :param delete_sql: a delete statement'''
    c = conn.cursor()
    c.execute(delete_sql)
    conn.commit()
    conn.close()

In [47]:
conn = create_connection(r'C:\Users\BPassini\Databases_Py\Ercot_SPP\LMP_DB.db')

In [12]:
sql_create_lmp_table = ''' Create Table if not exists ercot_hist_spp (
                                    DELIVERY_DATE text,
                                    DELIVERY_HOUR integer,
                                    DELIVERY_INTERVAL integer,
                                    REPEATED_HOUR_FLAG text,
                                    SETTLEMENT_POINT_NAME text,
                                    SETTLEMENT_POINT_TYPE text,
                                    SETTLEMENT_POINT_PRICE real);
                                '''

create_table(conn, sql_create_lmp_table)

In [30]:
conn.close()

In [28]:
%%time
conn = create_connection(r'C:\Users\BPassini\Databases_Py\Ercot_SPP\LMP_DB.db')

cursor = conn.cursor()

cursor.execute('''delete from ercot_hist_spp''')
print('table dropped')

conn.commit()
conn.close()

table dropped
Wall time: 17 ms


In [48]:
pd.read_sql_query('''select * from ercot_hist_spp where settlement_point_price is null''', conn)

,DELIVERY_DATE,DELIVERY_HOUR,DELIVERY_INTERVAL,REPEATED_HOUR_FLAG,SETTLEMENT_POINT_NAME,SETTLEMENT_POINT_TYPE,SETTLEMENT_POINT_PRICE


In [45]:
delete_rows(conn, delete_sql='''delete from ercot_hist_spp where settlement_point_price is null''')

In [42]:
max_date_df = pd.read_sql_query("Select max(DELIVERY_HOUR) from ercot_hist_spp where delivery_date = (select max(delivery_date) from ercot_hist_spp where settlement_point_price is not null)", conn)
max_date_df

,max(DELIVERY_HOUR)
0,24


In [33]:
df_1 = pd.read_sql_query("Select delivery_date, delivery_hour, settlement_point_name, avg(settlement_point_price) as spp from ercot_hist_spp group by delivery_date, delivery_hour, settlement_point_name ", conn)

In [34]:
df_1

,DELIVERY_DATE,DELIVERY_HOUR,SETTLEMENT_POINT_NAME,spp


In [ ]:
df = pd.read_sql_query("Select *  from ercot_hist_spp where settlement_point_price isnull", conn)

In [35]:
%%time
column_dict = {'Delivery Date':'DELIVERY_DATE'
               , 'Delivery Hour':'DELIVERY_HOUR'
               ,'Delivery Interval':'DELIVERY_INTERVAL'
               ,'Repeated Hour Flag':'REPEATED_HOUR_FLAG'
               ,'Settlement Point Name':'SETTLEMENT_POINT_NAME'
               ,'Settlement Point Type':'SETTLEMENT_POINT_TYPE'
               ,'Settlement Point Price':'SETTLEMENT_POINT_PRICE'}

for file in os.listdir(os.path.join(os.getcwd(),  'LMP_Zips')):
    if '.xlsx' in file:
        file = os.path.join(os.getcwd(),  'LMP_Zips', file)
        print(file)
        wb = xl.load_workbook(file, read_only=True)
        for sheet in wb.sheetnames:
            print(sheet)
            upload_sheet = pd.read_excel(file, sheet_name=sheet, engine='openpyxl')
            upload_sheet['Delivery Date'] = pd.to_datetime(upload_sheet['Delivery Date'])
            upload_sheet.rename(columns=column_dict, inplace=True)
            upload_sheet.to_sql(name='ercot_hist_spp', con=conn, if_exists='append', index=False)
        
            

C:\Users\BPassini\Github\Ercot_LMP_Scrape\LMP_Zips\rpt.00013061.0000000000000000.RTMLZHBSPP_2010.xlsx
Dec_1
C:\Users\BPassini\Github\Ercot_LMP_Scrape\LMP_Zips\rpt.00013061.0000000000000000.RTMLZHBSPP_2011.xlsx
Jan_1
Feb_2
Mar_3
Apr_4
May_5
Jun_6
Jul_7
Aug_8
Sep_9
Oct_10
Nov_11
Dec_12
C:\Users\BPassini\Github\Ercot_LMP_Scrape\LMP_Zips\rpt.00013061.0000000000000000.RTMLZHBSPP_2012.xlsx
Jan_1
Feb_2
Mar_3
Mar_4
Apr_5
May_6
May_7
Jun_8
Jul_9
Jul_10
Aug_11
Aug_12
Sep_13
Oct_14
Oct_15
Nov_16
Dec_17
Dec_18
C:\Users\BPassini\Github\Ercot_LMP_Scrape\LMP_Zips\rpt.00013061.0000000000000000.RTMLZHBSPP_2013.xlsx
Jan_1
Jan_2
Feb_3
Mar_4
Mar_5
Apr_6
May_7
May_8
Jun_9
Jul_10
Jul_11
Aug_12
Aug_13
Sep_14
Oct_15
Oct_16
Nov_17
Dec_18
Dec_19
C:\Users\BPassini\Github\Ercot_LMP_Scrape\LMP_Zips\rpt.00013061.0000000000000000.RTMLZHBSPP_2014.xlsx
Jan_1
Jan_2
Feb_3
Mar_4
Mar_5
Apr_6
May_7
May_8
Jun_9
Jul_10
Jul_11
Aug_12
Aug_13
Sep_14
Oct_15
Oct_16
Nov_17
Dec_18
Dec_19
C:\Users\BPassini\Github\Ercot_LMP_Scrape\LM